In [ ]:
import time
from math import factorial

import pandas as pd
import numpy as np

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# ExtraTreesClassifier

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_csv("input/train.csv")
train = reduce_mem_usage(train)
test = pd.read_csv("input/test.csv")
test = reduce_mem_usage(test)

train.drop(["row_id"] , axis = 1 , inplace = True)
test.drop(["row_id"] , axis = 1 , inplace = True)

In [ ]:
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'])

y = train["target"]
train.drop("target", axis=1, inplace=True)
# FEATURES = [col for col in train.columns if col not in ['row_id', "target"]]
# RANDOM_STATE = 12

In [ ]:
predictions, scores = [], []
 
k = StratifiedKFold(n_splits=10, random_state=228, shuffle=True)
for i, (trn_idx, val_idx) in enumerate(k.split(train, y)):
    
    X_train, X_val = train.iloc[trn_idx], train.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    model = ExtraTreesClassifier(n_estimators=1000, n_jobs=-1)
    model.fit(X_train, y_train)
    
    val_pred = model.predict(X_val)
    val_score = accuracy_score(y_val, val_pred)
    print(f'Fold {i+1} accuracy score: {round(val_score, 4)}')
    
    scores.append(val_score)
    predictions.append(model.predict_proba(test))
print('')    
print(f'Mean accuracy - {round(np.mean(scores), 4)}')

In [ ]:
# Postprocessing method: https://www.kaggle.com/code/ambrosm/tpsfeb22-02-postprocessing-against-the-mutants/notebook#The-surprise
y_proba = sum(predictions) / len(predictions)
y_proba += np.array([0, 0, 0.025, 0.045, 0, 0, 0, 0, 0, 0])
y_pred_tuned = le.inverse_transform(np.argmax(y_proba, axis=1))
pd.Series(y_pred_tuned, index=test.index).value_counts().sort_index() / len(test) * 100

In [ ]:
sub = pd.read_csv("input/sample_submission.csv")
sub['target'] = y_pred_tuned
sub.to_csv('submission_etc.csv', index=False)

# Radius Neighbors Classifier

In [ ]:
train_df = pd.read_csv("input/train.csv")
test_df = pd.read_csv("input/test.csv")

elements = [e for e in train_df.columns if e != 'row_id' and e != 'target']

# Convert the 10 bacteria names to the integers 0 .. 9
le = LabelEncoder()
train_df['target_num'] = le.fit_transform(train_df.target)

train_df.shape, test_df.shape

In [ ]:
# Idea from https://www.kaggle.com/code/ambrosm/tpsfeb22-01-eda-which-makes-sense
# Compute gcd and integer representations
def bias_of(s):
    """
    Bias is between 9.5e-7 and 2.4e-2. The sum of all biases is 1."""
    w = int(s[1:s.index('T')])
    x = int(s[s.index('T')+1:s.index('G')])
    y = int(s[s.index('G')+1:s.index('C')])
    z = int(s[s.index('C')+1:])
    return factorial(10) / (factorial(w) * factorial(x) * factorial(y) * factorial(z) * 4**10)

bias_vector = np.array([bias_of(col) for col in elements])

train_i = pd.DataFrame(((train_df[elements].values + bias_vector) * 1000000).round().astype(int), columns=elements, index=train_df.index)
test_i = pd.DataFrame(((test_df[elements].values + bias_vector) * 1000000).round().astype(int), columns=elements, index=test_df.index)

train_df['gcd'] = np.gcd.reduce(train_i[elements], axis=1)
test_df['gcd'] = np.gcd.reduce(test_i[elements], axis=1)


In [ ]:
# Select training samples with gcd=10000 (i.e. num_reads=100), drop duplicates and convert to integer
Z_tr = (train_i[(train_df.gcd == 10000)].drop_duplicates(elements) // 10000)
y_tr = train_df[(train_df.gcd == 10000)].drop_duplicates(elements).target_num

# Select test samples with gcd=10000 (i.e. num_reads=100) and convert to integer
Z_te = (test_i[(test_df.gcd == 10000)] // 10000)
Z_tr.shape, y_tr.shape, Z_te.shape

In [ ]:
%%time
# Convert Z_tr and Z_te to arrays X_tr and X_te of shape (n_samples, 100)
def transform(Z):
    ll = [] # list of lists which will be converted to a 2d array
    for i in range(len(Z)):
        l = [] # list which will be converted to a row of the new 2d array
        for j in range(Z.shape[1]):
            for k in range(Z.iloc[i, j]): l.append(j)
        ll.append(l)
    return np.array(ll)

X_tr = transform(Z_tr)
X_te = transform(Z_te)
X_tr.shape, X_te.shape

In [ ]:
# Predict with RadiusNeighborsClassifier
rnc = RadiusNeighborsClassifier(radius=18, weights='distance',
                                p=1, outlier_label=-1, n_jobs=-1)
rnc.fit(X_tr, y_tr)
y_pred = rnc.predict(X_te)
print('Unique predictions:', np.unique(y_pred))
print('Frequencies:', np.unique(y_pred, return_counts=True)[1])
print('Samples:', len(y_pred))
print('Predicted samples:', 
      len(y_pred) - np.unique(y_pred, return_counts=True)[1][0])

In [ ]:
# Read the ExtrTreesClassifier submission and merge it with our predictions
top_submission = pd.read_csv('submission_etc.csv')
test_df['target_num'] = le.transform(top_submission.target)
test_df.loc[(test_df.gcd == 10000), 'target_num'] = y_pred # can contain -1
test_df['target_num'] = np.where(test_df.target_num == -1,
                                 le.transform(top_submission.target),
                                 test_df.target_num)
test_df['target'] = le.inverse_transform(test_df.target_num.astype(int))
submission = test_df[['row_id', 'target']]
print('Modified:', (submission.target != top_submission.target).sum())
submission.to_csv('submission_radiusneighbors.csv', index=False)
submission
